# Follower Decision Algorithm (Fixed 2-Second Window)
This Jupyter notebook implements a simple algorithm to classify vehicle follower behavior (e.g., accelerate, decelerate, or do nothing) based on acceleration computed from speed data over fixed 2-second windows.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load your dataset here
# Example:
# data = pd.read_csv('your_data.csv')


In [ ]:
# Define the follower decision algorithm with a fixed 2-second window
def process_follower_decision_fixed_window(data):
    # Convert speed from ft/s to m/s
    data['speed_m_s'] = data['follower_speed'] * 0.3048

    # Initialize a column for decisions
    data['follower_decision'] = 'Do nothing'  # Default decision

    # Group by vehicle ID
    grouped = data.groupby('follower_vehicle_ID')

    for name, group in grouped:
        if len(group) < 2:
            print(f"Skipping vehicle {name} with insufficient data points.")
            continue

        # Calculate the acceleration
        time_interval = 0.1  # Assuming 0.1-second intervals
        acceleration_data_m_s2 = np.gradient(group['speed_m_s'], time_interval)

        # Process decisions in 2-second windows
        window_size = int(2 / time_interval)  # Number of points in 2 seconds
        decisions = []

        for i in range(0, len(acceleration_data_m_s2), window_size):
            window_acceleration = acceleration_data_m_s2[i:i+window_size]
            avg_acceleration = np.mean(window_acceleration)

            # Classify the decision
            if avg_acceleration > 0.5:
                decision = 'Accelerate'
            elif avg_acceleration < -0.5:
                decision = 'Decelerate'
            else:
                decision = 'Do nothing'

            decisions.extend([decision] * len(window_acceleration))

        # Update the decision column
        data.loc[group.index, 'follower_decision'] = decisions[:len(group)]

    return data

In [ ]:
# Apply the decision algorithm to the data
processed_data = process_follower_decision_fixed_window(data)
processed_data.head()